In [1]:
#pip install gurobipy==10

In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [3]:
# --------------- User-defined Parameters --------------- #
#M = 100000  # Maximum meals delivered by a bank in a two-week period, represents limits on food bank's logistics capabilities.
D = 50  # Maximum distance a person can travel from their home to a pantry in miles.
#P_min = 0.20  # Minimum percent of the FI population that must be served.
Budget = 1000000  # Total budget for the network.
Cost_per_meal_per_mile = 0.006  # Average cost per mile to transport food from a bank to a pantry.
Meals_Per_Person = 20 #Over a two week period, how many meals are given/served to that person
Minimum_Meals_Per_Pantry = 100  # Set the minimum number of meals to be delivered to each pantry

High_Risk = 0.5 #Minimum Proportion of FI population fed by food network for high risk counties
Medium_Risk = 0.30 #Minimum Proportion of FI population fed by food network for medium risk counties
Low_Risk = 0.2 #Minimum Proportion of FI population fed by food network for low risk counties

In [4]:
# Read the input data from the CSV files
banks_df = pd.read_csv('MSBankLatLong.csv')
pantries_df = pd.read_csv('MSPantryLatLong.csv')
demand_df = pd.read_csv('MS_City_Demand.csv')
bank_to_pantry_distance_df = pd.read_csv('BankToPantryDistanceMatrix.csv', index_col=0)
pantry_to_demand_distance_df = pd.read_csv('PantryToCityDistanceMatrix.csv', index_col=0)
county_risk_df = pd.read_csv("CountyRisk.csv", index_col=0)

In [5]:
def map_risk_to_p_min(risk):
    if risk == 'High':
        return High_Risk
    elif risk == 'Medium':
        return Medium_Risk
    elif risk == 'Low':
        return Low_Risk
    else:
        return None  # In case the risk is not one of the specified categories

# Apply the function to the 'Risk' column to create the 'P_Min' column
county_risk_df['P_Min'] = county_risk_df['Risk'].apply(map_risk_to_p_min)

# Display the updated dataframe
county_risk_df.head()

,latitude,longitude,Risk,FI Population 2021,2021 Cost Per Meal,2025 FI Population,2025 Cost Per Meal,P_Min
County,,,,,,,,
Adams,31.482943,-91.353498,Medium,"5,530",3.35,3715,3.79,0.3
Alcorn,34.880771,-88.580265,Medium,"6,260",3.41,6141,3.86,0.3
Amite,31.174397,-90.804431,Medium,"2,770",3.33,2736,3.77,0.3
Attala,33.086293,-89.581542,Medium,"3,310",3.14,2983,3.55,0.3
Benton,34.817286,-89.188475,Low,"1,170",3.24,970,3.67,0.2


In [6]:
# Initialize the model
model = gp.Model("Food_Distribution_Optimization")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-01


In [7]:
# Decision Variables

# x_bp: Amount of food delivered from bank b to pantry p.
x_bp = model.addVars(banks_df['Name'], pantries_df['Name'], vtype=GRB.INTEGER, name="x_bp")


# y_pc: Amount of FI persons fed from pantry p to demand node c.
y_pc = model.addVars(pantries_df["Name"], demand_df["City"], vtype=GRB.INTEGER, name="y_pc")

In [8]:
# Constraints
# Bank delivery constraint
#model.addConstrs((gp.quicksum(x_bp[b, p] for p in pantries_df['Name']) <= M for b in banks_df['Name']), name='bank_delivery')

# Distance constraint for pantries
model.addConstrs((pantry_to_demand_distance_df.loc[p, c]*y_pc[p,c] <= D*y_pc[p,c] for p in pantries_df['Name'] for c in demand_df['City']), name='pantry_distance')

# P_min constraint: Each city must have at least P_min percent of its food insecure people fed
#FI_population_df = demand_df[['City', 'FI Population 2025']].set_index('City')
#for c in demand_df['City']:
    #total_fi_population = FI_population_df.loc[c, 'FI Population 2025']
    #total_fi_population = int(total_fi_population.replace(',', ''))  
    #model.addConstr(gp.quicksum(y_pc[p, c] for p in pantries_df['Name']) >= P_min * total_fi_population, name=f'P_min_constraint_{c}')

# Aggregate food insecure population by county
county_fi_population_df = demand_df.groupby('County')['FI Population 2025'].sum()

# P_min constraint: Each county must have at least its specified P_min percent of its food insecure people fed
for county in county_fi_population_df.index:
    total_fi_population = county_fi_population_df.loc[county]
    p_min_for_county = county_risk_df.loc[county, 'P_Min']
    model.addConstr(gp.quicksum(y_pc[p, c] for p in pantries_df['Name'] for c in demand_df[demand_df['County'] == county]['City']) >= p_min_for_county * total_fi_population, name=f'P_min_constraint_{county}')
    
# Budget constraint
total_transport_cost = gp.quicksum(x_bp[b, p] * bank_to_pantry_distance_df.loc[b, p] * Cost_per_meal_per_mile 
                                   for b in banks_df['Name'] for p in pantries_df['Name'])
meal_cost_df = demand_df[['City', '2025 Cost Per Meal']].set_index('City')
total_procurement_cost = gp.quicksum(y_pc[p, c] * meal_cost_df.loc[c, '2025 Cost Per Meal'] 
                                     for p in pantries_df['Name'] for c in demand_df['City'])

total_cost = total_transport_cost + total_procurement_cost
model.addConstr(total_cost <= Budget, name='budget')

# Meals per person constraint
for p in pantries_df['Name']:
    for c in demand_df['City']:
        model.addConstr(y_pc[p, c] * Meals_Per_Person <= x_bp.sum('*', p), name=f'meals_per_person_constraint_{p}_{c}')

# Maximum FI population fulfillment constraint for each city
for c in demand_df['City']:
    fi_population = demand_df.loc[demand_df['City'] == c, 'FI Population 2025'].iloc[0]
    model.addConstr(gp.quicksum(y_pc[p, c] for p in pantries_df['Name']) <= fi_population, name=f'max_fulfillment_constraint_{c}')

# Max capacity constraint for each pantry
for p in pantries_df['Name']:
    max_capacity = pantries_df.loc[pantries_df['Name'] == p, 'Capacity'].iloc[0]
    model.addConstr(gp.quicksum(x_bp[b, p] for b in banks_df['Name']) <= max_capacity, name=f'max_capacity_constraint_{p}')

# Minimum meal delivery constraint for each pantry
for p in pantries_df['Name']:
    model.addConstr(gp.quicksum(x_bp[b, p] for b in banks_df['Name']) >= Minimum_Meals_Per_Pantry, name=f'min_meal_delivery_constraint_{p}')

# Supply and demand constraint
model.addConstrs((gp.quicksum(y_pc[p, c] for c in demand_df['City']) <= gp.quicksum(x_bp[b, p] for b in banks_df['Name']) for p in pantries_df['Name']), name='supply_demand')

# Demand fulfillment constraint
#model.addConstrs((y_pc[p, c] <= M for p in pantries_df['Name'] for c in demand_df['City']), name='demand_fulfillment')

# Non-negativity constraint
model.addConstrs((x_bp[b, p] >= 0 for b in banks_df['Name'] for p in pantries_df['Name']), name='non-negativity_x')
model.addConstrs((y_pc[p, c] >= 0 for p in pantries_df['Name'] for c in demand_df['City']), name='non-negativity_y')


{('Stewpot', 'Natchez'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Corinth'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Farmington'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Glen'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Rienzi'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Kossuth'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Gloster'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Liberty'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Crosby'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Kosciusko'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Ethel'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Sallis'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'McCool'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Ashland'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpot', 'Hickory Flat'): <gurobi.Constr *Awaiting Model Update*>,
 ('Stewpo

In [9]:
# Objective Function
model.setObjective(gp.quicksum(y_pc[p, c] for p in pantries_df['Name'] for c in demand_df["City"]), GRB.MAXIMIZE)


In [10]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 197511 rows, 65934 columns and 656010 nonzeros
Model fingerprint: 0x5a7c169d
Variable types: 0 continuous, 65934 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-04, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 1e+06]
Presolve removed 190978 rows and 59774 columns
Presolve time: 0.30s
Presolved: 6533 rows, 6160 columns, 36058 nonzeros
Variable types: 0 continuous, 6160 integer (0 binary)

Root relaxation: objective 2.053108e+05, 4944 iterations, 0.10 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 205310.842    0  121          - 205310.842      -     -    0s
H 

In [11]:
# Extract and print the results
solution_x_bp = model.getAttr('X', x_bp)
solution_y_pc = model.getAttr('X', y_pc)

for b, p in solution_x_bp.keys():
    if solution_x_bp[b, p] > 0:
        print(f"Bank {b} delivers {solution_x_bp[b, p]} meals to Pantry {p}.")


Bank Mississippi Food Network delivers 100.0 meals to Pantry Stewpot.
Bank Mississippi Food Network delivers 100.0 meals to Pantry Rose Hill Baptist Church.
Bank Mississippi Food Network delivers 100.0 meals to Pantry Pilgrim Baptist Church.
Bank Mississippi Food Network delivers 100.0 meals to Pantry First Assembly of God.
Bank Mississippi Food Network delivers 100.0 meals to Pantry 61 South Church of Christ.
Bank Mississippi Food Network delivers 100.0 meals to Pantry Catholic Charities.
Bank Mississippi Food Network delivers 100.0 meals to Pantry King’s Kabinet Food Pantry.
Bank Mississippi Food Network delivers 3760.0 meals to Pantry Helping Hands Ministries.
Bank Mississippi Food Network delivers 100.0 meals to Pantry Church of God In Christ.
Bank Mississippi Food Network delivers 1300.0 meals to Pantry Helping Hands of Cleveland.
Bank Mississippi Food Network delivers 100.0 meals to Pantry City of Port Gibson Emergency Shelter Team LLC.
Bank Mississippi Food Network delivers 5000

In [12]:
for p, c in solution_y_pc.keys():
    if solution_y_pc[p, c] > 0:
        print(f"Pantry {p} feeds {solution_y_pc[p, c]} FI persons in {c}.")

Pantry Stewpot feeds 5.0 FI persons in Natchez.
Pantry Stewpot feeds 5.0 FI persons in Gloster.
Pantry Stewpot feeds 5.0 FI persons in Crosby.
Pantry Stewpot feeds 5.0 FI persons in Bude.
Pantry Stewpot feeds 5.0 FI persons in Roxie.
Pantry Stewpot feeds 5.0 FI persons in Woodville.
Pantry Rose Hill Baptist Church feeds 5.0 FI persons in Natchez.
Pantry Rose Hill Baptist Church feeds 5.0 FI persons in Gloster.
Pantry Rose Hill Baptist Church feeds 5.0 FI persons in Crosby.
Pantry Rose Hill Baptist Church feeds 5.0 FI persons in Bude.
Pantry Rose Hill Baptist Church feeds 5.0 FI persons in Roxie.
Pantry Rose Hill Baptist Church feeds 5.0 FI persons in Meadville.
Pantry Rose Hill Baptist Church feeds 5.0 FI persons in Woodville.
Pantry Pilgrim Baptist Church feeds 5.0 FI persons in Natchez.
Pantry Pilgrim Baptist Church feeds 5.0 FI persons in Gloster.
Pantry Pilgrim Baptist Church feeds 5.0 FI persons in Crosby.
Pantry Pilgrim Baptist Church feeds 5.0 FI persons in Port Gibson.
Pantry P

In [13]:
#pip install folium

In [14]:
import matplotlib.pyplot as plt
import folium


# Initialize a map centered around Mississippi
map = folium.Map(location=[32.3547, -89.3985], zoom_start=7)  # Adjust zoom level as needed

# Draw pantry-to-city connections
for (p, c), value in solution_y_pc.items():
    if value > 0:
        pantry_location = (pantries_df.loc[pantries_df['Name'] == p, 'Latitude'].iloc[0], pantries_df.loc[pantries_df['Name'] == p, 'Longitude'].iloc[0])
        city_location = (demand_df.loc[demand_df['City'] == c, 'Latitude'].iloc[0], demand_df.loc[demand_df['City'] == c, 'Longitude'].iloc[0])
        folium.PolyLine([pantry_location, city_location], color="green", weight=2.5, opacity=1).add_to(map)

# Plot city locations
for idx, row in demand_df.iterrows():
    folium.CircleMarker((row['Latitude'], row['Longitude']), radius=5, color='green', fill=True).add_to(map)

# Plot pantry locations
for idx, row in pantries_df.iterrows():
    folium.CircleMarker((row['Latitude'], row['Longitude']), radius=5, color='red', fill=True).add_to(map)

# Draw bank-to-pantry connections
for (b, p), value in solution_x_bp.items():
    if value > 0:
        bank_location = (banks_df.loc[banks_df['Name'] == b, 'Latitude'].iloc[0], banks_df.loc[banks_df['Name'] == b, 'Longitude'].iloc[0])
        pantry_location = (pantries_df.loc[pantries_df['Name'] == p, 'Latitude'].iloc[0], pantries_df.loc[pantries_df['Name'] == p, 'Longitude'].iloc[0])
        folium.PolyLine([bank_location, pantry_location], color="blue", weight=2.5, opacity=1).add_to(map)

# Plot bank locations
for idx, row in banks_df.iterrows():
    folium.CircleMarker((row['Latitude'], row['Longitude']), radius=5, color='blue', fill=True).add_to(map)




# Save the map
map.save('map_2025.html')

/Users/CameronField/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [15]:
# Function to draw approximate Manhattan distance lines
def draw_manhattan_line(start_point, end_point, map_obj, color):
    # Intermediate point (keeping longitude of start and latitude of end)
    intermediate_point = (start_point[0], end_point[1])

    # Draw line from start to intermediate
    folium.PolyLine([start_point, intermediate_point], color=color, weight=2.5, opacity=1).add_to(map_obj)
    # Draw line from intermediate to end
    folium.PolyLine([intermediate_point, end_point], color=color, weight=2.5, opacity=1).add_to(map_obj)

# Initialize a map centered around Mississippi
map = folium.Map(location=[32.3547, -89.3985], zoom_start=7)  # Adjust zoom level as needed

# Plot city locations
for idx, row in demand_df.iterrows():
    folium.CircleMarker((row['Latitude'], row['Longitude']), radius=5, color='green', fill=True).add_to(map)

# Plot pantry locations
for idx, row in pantries_df.iterrows():
    folium.CircleMarker((row['Latitude'], row['Longitude']), radius=5, color='red', fill=True).add_to(map)

# Plot bank locations
for idx, row in banks_df.iterrows():
    folium.CircleMarker((row['Latitude'], row['Longitude']), radius=5, color='blue', fill=True).add_to(map)


# Draw pantry-to-city connections
for (p, c), value in solution_y_pc.items():
    if value > 0:
        pantry_location = (pantries_df.loc[pantries_df['Name'] == p, 'Latitude'].iloc[0], pantries_df.loc[pantries_df['Name'] == p, 'Longitude'].iloc[0])
        city_location = (demand_df.loc[demand_df['City'] == c, 'Latitude'].iloc[0], demand_df.loc[demand_df['City'] == c, 'Longitude'].iloc[0])
        draw_manhattan_line(pantry_location, city_location, map, color="green")

# Draw bank-to-pantry connections
for (b, p), value in solution_x_bp.items():
    if value > 0:
        bank_location = (banks_df.loc[banks_df['Name'] == b, 'Latitude'].iloc[0], banks_df.loc[banks_df['Name'] == b, 'Longitude'].iloc[0])
        pantry_location = (pantries_df.loc[pantries_df['Name'] == p, 'Latitude'].iloc[0], pantries_df.loc[pantries_df['Name'] == p, 'Longitude'].iloc[0])
        draw_manhattan_line(bank_location, pantry_location, map, color="blue")

# Save the map
map.save('manhattan_map_2025.html')

In [16]:
#pip install geopandas

In [17]:
import pandas as pd
import folium
import geopandas as gpd

# Step 1: Create a DataFrame for FI Fed Solution
# Assuming solution_y_pc contains the solution data
fi_fed_per_city = pd.DataFrame([(p, c, solution_y_pc[p, c]) for (p, c) in solution_y_pc], columns=['Pantry', 'City', 'FI_Fed'])
fi_fed_per_city = fi_fed_per_city.groupby('City')['FI_Fed'].sum().reset_index()

# Step 2: Map Cities to Counties using demand_df
city_to_county = demand_df[['City', 'County']].drop_duplicates().set_index('City')
fi_fed_per_city = fi_fed_per_city.join(city_to_county, on='City')

# Step 3: Aggregate FI Fed Data by County
fi_fed_per_county = fi_fed_per_city.groupby('County')['FI_Fed'].sum()

# Step 4: Calculate the Percentage Fed
# Assuming demand_df has total FI population per county
total_fi_population_per_county = demand_df.groupby('County')['FI Population 2025'].sum()
percentages = (fi_fed_per_county / total_fi_population_per_county * 100).fillna(0)

# Function to determine the color based on the percentage fed
def get_color(percentage):
    if percentage >= 70:
        return 'green'
    elif percentage >= 40:
        return 'orange'
    elif percentage == 0:
        return 'gray'
    else:
        return 'red'

# Add labels to each county
def add_labels(feature):
    county_name = feature['properties']['name']
    fi_population = total_fi_population_per_county.get(county_name, 0)
    percent_fed = percentages.get(county_name, 0)
    return f"{county_name}<br>FI Population: {fi_population}<br>Percent Fed: {percent_fed:.2f}%"

# Load the county geospatial data
counties_geojson = 'mississippi-with-county-boundaries_1108.geojson'
counties = gpd.read_file(counties_geojson)

# Initialize a map centered around Mississippi
ms_map = folium.Map(location=[32.3547, -89.3985], zoom_start=7)

# Add GeoJson to the map with color coding and labels
folium.GeoJson(
    counties_geojson,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': get_color(percentages.get(feature['properties']['name'], 0)),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name'],
        aliases=['County: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 4px;") 
    )
).add_to(ms_map)

# Add labels for each county
for _, county in counties.iterrows():
    # Directly access the county name
    county_name = county['name']  # Adjust 'name' based on the actual column name in the GeoDataFrame
    centroid = county['geometry'].centroid
    label = f"{percentages.get(county_name, 0):.2f}%"
    folium.Marker(
        location=[centroid.y, centroid.x],
        icon=folium.DivIcon(html=f'<div style="font-size: 12pt">{label}</div>'),
    ).add_to(ms_map)

# Save the map
ms_map.save('ms_counties_map_2025.html')
